In [2]:
!pip install datasets optuna scikit-learn

In [ ]:
import gdown

file_id = '1Y9QFfJRUIEbrvLtQRZgX6W4O3z1IzDhc'
gdown.download(f'https://drive.google.com/uc?id={file_id}', '/content/DL_train_test_data.xlsx', quiet=False)

Downloading...From: https://drive.google.com/uc?id=1Y9QFfJRUIEbrvLtQRZgX6W4O3z1IzDhcTo: /content/DL_train_test_data.xlsx100%|██████████| 25.9M/25.9M [00:00<00:00, 52.7MB/s]

'/content/DL_train_test_data.xlsx'

In [3]:
# Load data
from transformers import AutoTokenizer, AlbertForSequenceClassification
from torch.nn import CrossEntropyLoss
from sklearn.metrics import classification_report, accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import optuna
from tqdm.notebook import tqdm
import numpy as np
from torch.optim import AdamW


model_name = 'albert-base-v2'  # Use ALBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
df = pd.read_excel('/kaggle/input/dl-train-test/DL_train_test_data.xlsx')

# Encode severity levels
df['severity_label'] = df['severity_code']
print(df['severity_label'].unique())  # Ensure labels range from 0 to 6

# Combine short and long descriptions into a single text field
df['text'] = df['short_description'].fillna('') + ' ' + df['long_description'].fillna('')

# Tokenize data
tokenized_data = tokenizer(
    df['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[2 6 1 4 5 0]


In [6]:

class BugDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


labels = df['severity_label'].values
dataset = BugDataset(tokenized_data, labels)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlbertForSequenceClassification.from_pretrained(model_name, num_labels=7)
model.to(device)

/tmp/ipykernel_30/999302903.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=

In [7]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import WeightedRandomSampler
from torch.optim import AdamW
from tqdm import tqdm


# Training parameters
epochs = 10
# batch_size = best_hyperparams['batch_size']
# learning_rate = best_hyperparams['learning_rate']
batch_size = 16
learning_rate = 1e-6

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Validation
    model.eval()
    val_loss = 0
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    avg_val_loss = val_loss / len(val_loader)
    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')  # Use 'weighted' for imbalanced classes

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(classification_report(true_labels, preds))

100%|██████████| 2568/2568 [08:26<00:00,  5.07it/s]


Epoch 1/10 | Train Loss: 0.6761923181154182


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 0.6077
Validation Accuracy: 0.8415
Validation F1 Score: 0.7706
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       230
           2       0.84      1.00      0.91      8631
           4       0.00      0.00      0.00       665
           5       0.78      0.03      0.05       547
           6       0.00      0.00      0.00       197

    accuracy                           0.84     10270
   macro avg       0.32      0.21      0.19     10270
weighted avg       0.75      0.84      0.77     10270



100%|██████████| 2568/2568 [08:26<00:00,  5.07it/s]


...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 0.7515
Validation Accuracy: 0.7984
Validation F1 Score: 0.7733
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       230
           2       0.86      0.92      0.89      8631
           4       0.14      0.13      0.14       665
           5       0.35      0.25      0.29       547
           6       0.00      0.00      0.00       197

    accuracy                           0.80     10270
   macro avg       0.27      0.26      0.26     10270
weighted avg       0.75      0.80      0.77     10270



100%|██████████| 2568/2568 [08:25<00:00,  5.08it/s]


Epoch 10/10 | Train Loss: 0.2822710892266806
Validation Loss: 0.7875
Validation Accuracy: 0.7993
Validation F1 Score: 0.7731
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       230
           2       0.86      0.93      0.89      8631
           4       0.15      0.13      0.14       665
           5       0.35      0.23      0.28       547
           6       0.00      0.00      0.00       197

    accuracy                           0.80     10270
   macro avg       0.27      0.26      0.26     10270
weighted avg       0.75      0.80      0.77     10270



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
# Save the model
torch.save(model.state_dict(), 'optimized_model.pth')
from google.colab import files
files.download('optimized_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import WeightedRandomSampler
from torch.optim import AdamW
from tqdm import tqdm


# Training parameters
epochs = 10
# batch_size = best_hyperparams['batch_size']
# learning_rate = best_hyperparams['learning_rate']
batch_size = 16
learning_rate = 1e-6

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Validation
    model.eval()
    val_loss = 0
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    avg_val_loss = val_loss / len(val_loader)
    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')  # Use 'weighted' for imbalanced classes

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(classification_report(true_labels, preds))